In [1]:
# proper

In [ ]:
import sys
import argparse
import torch
import cv2
import time
import re
import numpy as np
from ultralytics import YOLO
import math
from typing import List

model_path = '/home/agrotech/deep-gauge-master/Reading_Circular_Analogue_Gauges_using_Digital_Image_Processing/model_weights/pose_gauge.pt'

theta_A = 0
theta_B = 0

def detectx(frame, model):
    frame = [frame]
    print(f"[INFO] Detecting. . . ")
    results = model(frame)
    print(results.xyxyn[0])
    print(results.xyxyn[0][:, -1])
    print(results.xyxyn[0][:, :-1])
    labels, coordinates = results.xyxyn[0][:, -1], results.xyxyn[0][:, :-1]
    return labels, coordinates

def plot_boxes(results, frame, classes):
    labels, cord = results
    n = len(labels)
    x_shape, y_shape = frame.shape[1], frame.shape[0]
    print(f"[INFO] Total {n} detections. . . ")
    print(f"[INFO] Looping through all detections. . . ")

    for i in range(n):
        row = cord[i]
        if row[4] >= 0.37:
            print(f"[INFO] Extracting BBox coordinates. . . ")
            x1, y1, x2, y2 = int(row[0]*x_shape), int(row[1]*y_shape), int(row[2]*x_shape), int(row[3]*y_shape)
            text_d = classes[int(labels[i])]
            print(text_d)
            coords = [x1, y1, x2, y2]
            plate_num, plantnum1 = calculate_pointer_angle(img=frame, coords=coords)
            minimum, maximum, argument = gauge_read(text_d)

            if argument == 'Gauge not in Dictionary':
                text_d = 'Gauge not in Dictionary'

            print(f"minium is: {minimum:.1f}")
            print(f"maximum is: {maximum:.1f}")

            value = minimum + (plate_num / plantnum1) * maximum

            print(f"Gauge reading is {value:.1f}")

            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.rectangle(frame, (x1, y1-20), (x2, y1), (0, 255, 0), -1)
            cv2.putText(frame, f"{value:.1f}", (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            cv2.putText(frame, f"{text_d}", (x1, y1-30), cv2.FONT_HERSHEY_SIMPLEX, .75, (255, 255, 0), 1)

    return frame

def calculate_pointer_angle(img, coords):
    xmin, ymin, xmax, ymax = coords
    nplate = img[int(ymin):int(ymax), int(xmin):int(xmax)]
    model1 = YOLO(model_path)
    results = model1(img)
    theta1 = None
    theta2 = None

    for result in results:
        keypoints_set = result.keypoints.xy.cpu().numpy()
        print(keypoints_set)

        if keypoints_set.size == 0:
            print('No Key Point Detected')
            theta1 = 0
            theta2 = 45
            break

        if keypoints_set.shape[1] < 4:
            print('Not enough keypoints detected')
            break

        A, B, C, D = keypoints_set[0][0], keypoints_set[0][1], keypoints_set[0][3], keypoints_set[0][2]
        print(f"A is {A[1]}")
        print(f"B is {B[1]}")
        print(f"C is {C[1]}")

        theta1 = compute_angle(A, B, C)
        theta2 = 360 - compute_angle(B, D, C)
        print(f"Angle between keypoints A, B, and C is: {theta1:.2f} degrees")
        print(f"Angle between keypoints B, C, and D is: {theta2:.2f} degrees")

    return theta1, theta2

def compute_angle(A, B, C):
    a = [A[0] - B[0], A[1] - B[1]]
    b = [C[0] - B[0], C[1] - B[1]]
    c = [A[0] - C[0], A[1] - C[1]]

    ma = math.sqrt(a[0] ** 2 + a[1] ** 2)
    mb = math.sqrt(b[0] ** 2 + b[1] ** 2)
    mc = math.sqrt(c[0] ** 2 + c[1] ** 2)

    print(ma)
    print(mb)
    print(mc)

    cosB = (ma ** 2 + mc ** 2 - mb ** 2) / (2 * ma * mc)
    angle = math.acos(cosB)
    angle_deg = math.degrees(angle)

    return angle_deg

def main(img_path=None):
    print(f"[INFO] Loading V5 model... ")
    model = torch.hub.load('/home/agrotech/deep-gauge-master/Reading_Circular_Analogue_Gauges_using_Digital_Image_Processing/yolov5-master/yolov5/yolov5', 'custom', source='local', path='demo_20feb_v1.pt', force_reload=True)
    classes = model.names

    if img_path is not None:
        print(f"[INFO] Working with image: {img_path}")
        img_out_name = f"./output/result_{img_path.split('/')[-1]}"
        frame = cv2.imread(img_path)
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = detectx(frame, model=model)
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        frame = plot_boxes(results, frame, classes=classes)

        while True:
            frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
            cv2.imwrite(f"output.jpg", frame)
            break

def gauge_read(text_d):
    gauge_limits = {
        'Pressure Gauge2': (0, 16, None),
        'Pressure Gauge3': (0, 15, None),
        'Pressure Gauge4': (0, 10, None),
        'Pressure Gauge1': (0, 28, None),
        'Fire Pressure Gauge1': (0, 15, None),
        'SG6 Pressure Gauge1': (0, 950, None)
    }

    result = gauge_limits.get(text_d, (0, 0, "Gauge not in Dictionary"))
    print(text_d)

    return result

if __name__ == "__main__":
    parser = argparse.ArgumentParser(description='Process some images')
    parser.add
